In [4]:
library(lme4)
library(lmerTest)
library(stringr)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [5]:
data_profile <- read.csv('/Users/m221138/RA_plasma_metabolites/data/67_met_every_confounds_v2.csv', row.names = 1, header=TRUE, sep=",")
data_profile <- as.data.frame(t(data_profile))
output_drug_file <- '/Users/m221138/RA_plasma_metabolites/data/67_met_every_drug_v3.logistic.results.tsv'
output_confounding_file <- '/Users/m221138/RA_plasma_metabolites/data/67_met_every_confounds_v3.logistic.results.tsv'
output_confounding_v2_file <- '/Users/m221138/RA_plasma_metabolites/data/9_met_confounds_v3.logistic.results.tsv'


In [6]:
head(data_profile)

,patient_dummyID,SEX,AGE,mtx,csdmard,tnfi,ntnfi,pred,smoke,crp,⋯,100002911,1489,100020361,100001104,100000269,100020866,100001510,100001767,171,344
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ST.00010415_t1,0,1,75,1,0,0,0,0,1,0,⋯,1.6174,5.8065,1.5960,2.3718,0.7689,4.2787,2.8148,1.5577,0.0243,1.3874
ST.00010415_t2,0,1,76,1,0,0,0,0,1,0,⋯,0.9193,0.5310,1.1545,1.4633,0.9511,4.7451,2.6375,0.8981,0.2159,1.3073
ST.00010421_t1,1,1,70,1,0,1,0,1,1,1,⋯,1.3136,7.5976,1.3427,1.1518,1.3642,1.7638,0.7221,3.1478,0.1789,1.3242
ST.00010421_t2,1,1,71,1,0,1,0,1,1,0,⋯,3.3806,0.4750,0.9460,0.8480,0.7717,1.4401,1.4132,0.5696,0.4390,1.1715
ST.00011458_t1,2,0,69,0,1,0,0,0,0,0,⋯,0.3923,2.7211,1.1114,0.8064,2.3825,0.9498,0.9553,1.2109,0.1584,1.0916
ST.00011458_t2,2,0,69,0,1,0,0,0,0,0,⋯,2.5219,0.4712,1.0083,0.8661,1.2101,0.9348,1.5982,0.3697,1.0086,1.5167


In [7]:
NUM_CHEM_ID <- ncol(data_profile)

In [21]:
if (file.exists(output_confounding_file)){
    file.remove(output_confounding_file)
}

if (file.exists(output_confounding_v2_file)){
    file.remove(output_confounding_v2_file)
}

cat("chemID\tEveryConfound(re)\tAGE(re)\tSEX(re)\tsmoke(re)\tMTX(re)\tPRED(re)\tntnfi(re)\ttnfi(re)\tcsdmard(re)\tcrp(re)\n", file=output_confounding_file,append=TRUE)
cat("chemID\tpvalue\n", file=output_confounding_v2_file,append=TRUE)
for (i in 1:NUM_CHEM_ID){
    if (i > 10){
        
    chemID <- colnames(data_profile)[i]
    chem_profile <- data_profile[,i]
        
    #EVERYTHING
    all_m <- glmer(crp ~ chem_profile + SEX + mtx + smoke + pred + ntnfi + tnfi + csdmard + AGE + (1|patient_dummyID), family = binomial, data = data_profile)    
    pval_list <- coef(summary(all_m))[,4]
    all_p <- pval_list[2]
        
    age_m <- lmer(AGE ~ chem_profile + (1|patient_dummyID), data = data_profile)
    pval_list <- coef(summary(age_m))[,4]
    age_p <- pval_list[2]
        
    sex_m <- glmer(SEX ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(sex_m))[,4]
    sex_p <- pval_list[2]
        
    smoke_m <- glmer(smoke ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(smoke_m))[,4]
    smoke_p <- pval_list[2]
        
    mtx_m <- glmer(mtx ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(mtx_m))[,4]
    mtx_p <- pval_list[2]
        
    pred_m <- glmer(pred ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(pred_m))[,4]
    pred_p <- pval_list[2]
        
    ntnfi_m <- glmer(ntnfi ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(ntnfi_m))[,4]
    ntnfi_p <- pval_list[2]   
        
    tnfi_m <- glmer(tnfi ~ chem_profile + (1|patient_dummyID),family = binomial,  data = data_profile)
    pval_list <- coef(summary(tnfi_m))[,4]
    tnfi_p <- pval_list[2]
           
    csdmard_m <- glmer(csdmard ~ chem_profile + (1|patient_dummyID), family = binomial, data = data_profile)
    pval_list <- coef(summary(csdmard_m))[,4]
    csdmard_p <- pval_list[2]
        
    crp_m <- glmer(crp ~ chem_profile + (1|patient_dummyID), family =binomial, data = data_profile)
    pval_list <- coef(summary(crp_m))[,4]
    crp_p <- pval_list[2]
            
    output_string <- paste(chemID, all_p, age_p, sex_p, smoke_p, mtx_p, pred_p, ntnfi_p, tnfi_p, csdmard_p, crp_p, "\n", sep="\t")
    cat(output_string, file=output_confounding_file,append=TRUE)
     
#     if (chemID == 803){
#     task1_m <- lmer(chem_profile ~ crp + smoke + mtx + ntnfi + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }
#     if (chemID == 503){
#     task1_m <- lmer(chem_profile ~ crp + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }    

#     if (chemID == 1442){
#     task1_m <- lmer(chem_profile ~ crp + SEX + csdmard + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }
        
#     if (chemID == 100002945){
#     task1_m <- lmer(chem_profile ~ crp + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     } 
        
#     if (chemID == 100001197){
#     task1_m <- lmer(chem_profile ~ crp + smoke + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }        
        
#     if (chemID == 100015831){
#     task1_m <- lmer(chem_profile ~ crp + pred + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }    
        
#     if (chemID == 1087){
#     task1_m <- lmer(chem_profile ~ crp + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }        

#     if (chemID == 100000010){
#     task1_m <- lmer(chem_profile ~ crp + tnfi + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }   
        
#     if (chemID == 100001577){
#     task1_m <- lmer(chem_profile ~ crp + SEX + pred + (1|patient_dummyID), data = data_profile)    
#     task1_p <- anova(task1_m)$Pr[1]
#     output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
#     cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
#     }   
     
}}

print ("Done")

[1] TRUE

[1] TRUE

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.379825 (tol = 0.001, component 1)”


ERROR: Error in eval(family$initialize, rho): y values must be 0 <= y <= 1
